In [8]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [10]:
## Use inbuilt tool for wikipedia
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [11]:
## Use inbuilt tool for arxiv
api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper)
arxiv.name

'arxiv'

In [12]:
tools=[wiki,arxiv]

In [14]:
wiki.invoke("Who is Narendra modi")

'Page: List of awards and honours received by Narendra Modi\nSummary: This is a comprehensive list of awards, honours, and recognitions received by Narendra Modi, the 14th and current Prime Minister of India since 2014. As of 2025, Modi has received mo'

In [27]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db=FAISS.from_documents(documents,embedding)
retriever=vector_db.as_retriever()

In [24]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith-serch","Search any information about langsmith")
retriever_tool

Tool(name='langsmith-serch', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001CC21E89750>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001CC243483D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001CC21E897E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001CC243483D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), doc

In [44]:
tools=[retriever_tool,wiki,arxiv]

In [31]:
from langchain_groq import ChatGroq
llm =ChatGroq(model_name="Llama3-8b-8192")

In [32]:
llm.invoke("HII")

AIMessage(content='Hii! How are you doing today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_time': 0.023064832, 'prompt_time': 0.004119878, 'queue_time': 0.265775353, 'total_time': 0.02718471}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0aae01a-429c-4e17-a44f-c8e6ffb0f40b-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [34]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [45]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [46]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)

In [49]:
agent_executor.invoke({"input": "Researchpaper attention is all we need"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Researchpaper attention is all we need'}`


Published: 2017-09-19
Title: Exploring Human-like Attention Supervision in Visual Question Answering
Authors: Tingting Qiao, Jianfeng Dong, Duanqing Xu
Summary: Attention mechanisms have been widely applied in the Visual Question
Answering (VQA) task
Invoking: `wikipedia` with `{'query': 'Attention mechanism'}`


Page: Attention (machine learning)
Summary: In machine learning, attention is a  method that determines the importance of each component in a sequence relative to the other components in that sequence. In natural language processing, importance is reHere's a helpful response:

The concept of attention is rooted in the idea that not all parts of a sequence are equally important. In the context of visual question answering, attention mechanisms help the model focus on the relevant parts of the image to answer the question. The paper "Exploring Human-like Attent

{'input': 'Researchpaper attention is all we need',
 'output': 'Here\'s a helpful response:\n\nThe concept of attention is rooted in the idea that not all parts of a sequence are equally important. In the context of visual question answering, attention mechanisms help the model focus on the relevant parts of the image to answer the question. The paper "Exploring Human-like Attention Supervision in Visual Question Answering" uses attention mechanisms to improve the performance of VQA models.'}